In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
netflix= pd.read_csv("netflix_titles.csv")
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [3]:
filledna=netflix.fillna('')
filledna.head(2)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...


In [8]:
def clean_data(x):
        x=str(x)
        return str.lower(x.replace(" ", ""))
def create_soup(x):
    return x['title']+ ' ' + x['director'] + ' ' + x['cast'] + ' ' +x['listed_in']+' '+ x['description']+' '+x["type"]+" "+x["rating"]

In [5]:
features=['title','director','cast','listed_in','description',"type","rating"]
filledna=filledna[features]

In [6]:
for feature in features:
    filledna[feature] = filledna[feature].apply(clean_data)
    
filledna.head(2)

,title,director,cast,listed_in,description,type,rating
0,3%,,"joãomiguel,biancacomparato,michelgomes,rodolfo...","internationaltvshows,tvdramas,tvsci-fi&fantasy",inafuturewheretheeliteinhabitanislandparadisef...,tvshow,tv-ma
1,7:19,jorgemichelgrau,"demiánbichir,héctorbonilla,oscarserrano,azalia...","dramas,internationalmovies","afteradevastatingearthquakehitsmexicocity,trap...",movie,tv-ma


In [9]:
filledna['soup'] = filledna.apply(create_soup, axis=1)
filledna.head(2)

,title,director,cast,listed_in,description,type,rating,soup
0,3%,,"joãomiguel,biancacomparato,michelgomes,rodolfo...","internationaltvshows,tvdramas,tvsci-fi&fantasy",inafuturewheretheeliteinhabitanislandparadisef...,tvshow,tv-ma,"3% joãomiguel,biancacomparato,michelgomes,rod..."
1,7:19,jorgemichelgrau,"demiánbichir,héctorbonilla,oscarserrano,azalia...","dramas,internationalmovies","afteradevastatingearthquakehitsmexicocity,trap...",movie,tv-ma,"7:19 jorgemichelgrau demiánbichir,héctorbonill..."


In [10]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from annoy import AnnoyIndex

count = TfidfVectorizer(stop_words='english')
count_matrix = count.fit_transform(filledna['soup'])
count_matrix=count_matrix.toarray()

In [11]:
filledna=filledna.reset_index()
indices = pd.Series(filledna.index, index=filledna['title'])

In [12]:
print(count_matrix[2])

[0. 0. 0. ... 0. 0. 0.]


In [14]:
f= len(count_matrix)
print(f)
t = AnnoyIndex(68322, 'angular')
for i in range(f):
    t.add_item(i,count_matrix[i])
t.build(100,n_jobs=-1)
t.save('annoy100.ann')
print("done")

7787
done


In [15]:
def get_recommendations_new(title):
    
    title=title.replace(' ','').lower()
    idx = indices[title]
    print(idx)
    # Get the pairwsie similarity scores of all movies with that movie
    u = AnnoyIndex(68322, 'angular')
    u.load('annoy100.ann') # super fast, will just mmap the file
    similar=u.get_nns_by_item(idx, 10)
    return netflix['title'].iloc[similar]

In [19]:
print(get_recommendations_new("Bojack Horseman"))

1025
1025                                      BoJack Horseman
1026    BoJack Horseman Christmas Special: Sabrina's C...
7450                                       W/ Bob & David
621                    Aunty Donna's Big Ol' House of Fun
6491                                       The Informant!
6316            The Drunk and on Drugs Happy Funtime Hour
4760                                        Paquita Salas
1966                                           El testigo
5669                                       Slobby's World
6830    The Road to El Camino: Behind the Scenes of El...
Name: title, dtype: object
